## Chaine de traitement en Python pour adapter le moteur de reconnaissance d'entités nommées de Spacy par entrainement. 

Il s'agit de :


1.   annoter automatiquement en noms de lieux (LOC) et de personnes (PERS) un extrait du texte "Bel Ami" de Maupassant en s'appuyant sur le modèle pour le français "fr_core_news_sm" proposé par Spacy ;
2.   transférer le texte annoté vers l'outil d'annotation TagTog via leur API REST afin de corriger manuellement les annotations créées automatiquement;
3.   Corriger manuellement ces annotations en vue de créer un jeu  de données d'entrainement ;
4.   Formatter et intégrer les annotations corrigées dans Spacy en tant que jeu de données d'entrainement ;
5.  Initialiser un modèle qui sera entrainé de manière incrémentale à partir du jeu d'entrainement ;
6.  Initialiser l'entrainement avec les paramètres définis dans le fichier .cfg et entrainement par le biais du CLI de Spacy ;
7.  Test du nouveau modèle sur un autre extrait de "Bel Ami" de Maupassant.





**Prérequis : Installation ou mise à jour de Spacy et téléchargement du modèle**

In [2]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    2.2.4                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.12                        
Models           en                            



In [3]:
!pip install -U pip setuptools wheel
!pip install -U spacy

     |████████████████████████████████| 2.1 MB 14.1 MB/s 
     |████████████████████████████████| 1.1 MB 60.1 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.0/452.0 KB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.2/628.2 KB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 KB 19.7 MB/s eta 0:00:00
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installa

In [4]:
!python -m spacy download fr_core_news_sm  

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: fr-core-news-sm
    Found existing installation: fr-core-news-sm 2.2.5
    Uninstalling fr-core-news-sm-2.2.5:
      Successfully uninstalled fr-core-news-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


**Chargement d'un extrait du premier chapiture de Bel Ami de Maupassant.**
A télécharger et enregistrer sur votre ordinateur à partir de : 
https://github.com/cvbrandoe/coursTAL/blob/master/2022/belAmi_01-01.txt 

In [7]:
from google.colab import files
uploaded = files.upload()

Saving belAmi_01-01.txt to belAmi_01-01.txt


**1. Annotation automatiquement en noms de lieux (LOC) et de personnes (PERS) du texte en entrée en s'appuyant sur le modèle pour le français "fr_core_news_sm" proposé par Spacy**

In [13]:
import spacy
import json
import requests
import os

def get_class_id(label):
  """
  Translates the spaCy label id into the tagtog entity type id
  - label: spaCy label id
  """
  choices = {'PER': 'e_1', 'LOC': 'e_2', 'GPE': 'e_2'}
  return choices.get(label, None)

def get_entities(spans, pipeline):
  """
  Translates a tuple of named entity Span objects (https://spacy.io/api/span) into a 
  list of tagtog entities (https://docs.tagtog.net/anndoc.html#ann-json). Each entity is
  defined by the entity type ID (classId), the part name where the annotation is (part),
  the entity offsets and the confidence (annotation status, who created it and probabilty).
  - spans: the named entities in the spaCy doc
  - pipeline: trained pipeline name
  """
  default_prob = 1
  default_part_id = 's1v1'
  default_state = 'pre-added'
  tagtog_entities = []
  for span in spans:
    class_id = get_class_id(span.label_)
    if class_id is not None:
      tagtog_entities.append( {
        'classId': class_id,
        'part': default_part_id,
        'offsets':[{'start': span.start_char, 'text': span.text}],
        'confidence': {'state': default_state,'who': ['ml:' + pipeline],'prob': default_prob},
        'fields':{},
        # this is related to the kb_id (knowledge base ID) field from the Span spaCy object
        'normalizations': {}} )
  return tagtog_entities


In [14]:

#text = "Quand Georges Duroy parvint au boulevard, il s’arrêta encore, indécis sur ce qu’il allait faire. Il avait envie maintenant de gagner les Champs-Élysées et l’avenue du bois de Boulogne pour trouver un peu d’air frais sous les arbres."
myfile = open("belAmi_01-01.txt").read().replace("’","'")
pipeline = 'fr_core_news_sm' 
nlp = spacy.load(pipeline)
doc = nlp(myfile)

def show_ents(doc): 
    if doc.ents: 
        for ent in doc.ents: 
            print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
            print("\n")             
show_ents(doc)


PREMIÈRE PARTIE - 0 - 15 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


Georges Duroy - 95 - 108 - PER - Named person or family.


prix fixe - 701 - 710 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


rue Notre-Dame-de-Lorette - 1378 - 1403 - LOC - Non-GPE locations, mountain ranges, bodies of water


Grand - 2082 - 2087 - PER - Named person or family.


Paris - 2459 - 2464 - LOC - Non-GPE locations, mountain ranges, bodies of water


Georges Duroy - 2963 - 2976 - PER - Named person or family.


Champs-Élysées - 3094 - 3108 - LOC - Non-GPE locations, mountain ranges, bodies of water


bois de Boulogne - 3124 - 3140 - LOC - Non-GPE locations, mountain ranges, bodies of water


Venez - 3651 - 3656 - LOC - Non-GPE locations, mountain ranges, bodies of water


joli garçon - 3672 - 3683 - PER - Named person or family.


jaunes - 4478 - 4484 - LOC - Non-GPE locations, mountain ranges, bodies of water


verts - 4

**2. transfer du texte annoté vers l'outil d'annotation TagTog via leur API REST afin de corriger manuellement les annotations**

Au préalable, il faut créer un compte utilisateur et un projet dans TagTog (https://www.tagtog.net/) et modifier les informations de connexion ci-dessous.


In [18]:
# Set the credentials at tagtog and project name
MY_USERNAME = 'ACHANGER'
MY_PASSWORD = 'ACHANGER'
MY_PROJECT = 'ACHANGER'

# API authentication
tagtogAPIUrl = "https://www.tagtog.net/-api/documents/v1"
auth = requests.auth.HTTPBasicAuth(username=MY_USERNAME, password=MY_PASSWORD)

# Initialize ann.json (specification: https://docs.tagtog.net/anndoc.html#ann-json)
annjson = {}
# Set the document as not confirmed, an annotator will manually confirm whether the annotations are correct
annjson['anncomplete'] = False
annjson['metas'] = {}
annjson['relations'] = []                      
# Transform the spaCy entities into tagtog entities
annjson['entities'] = get_entities(doc.ents, pipeline)

print(myfile)
print(json.dumps(annjson))

# Parameters for the API call 
# see https://docs.tagtog.net/API_documents_v1.html#examples-import-pre-annotated-plain-text-file
params = {'owner': MY_USERNAME, 'project': MY_PROJECT, 'output': 'null', 'format': 'default-plus-annjson'}
# Pre-annotated document composed of the content and the annotations
files=[('BelAmi_extrait0101.txt', myfile), ('BelAmi_extrait0101.ann.json', json.dumps(annjson))]
# POST request to send the pre-annotated document
response = requests.post(tagtogAPIUrl, params=params, auth=auth, files=files)

print(response.text)

PREMIÈRE PARTIE.



I



Quand la caissière lui eut rendu la monnaie de sa pièce de cent sous, Georges Duroy sortit du restaurant.

Comme il portait beau, par nature et par pose d'ancien sous-officier, il cambra sa taille, frisa sa moustache d'un geste militaire et familier, et jeta sur les dîneurs attardés un regard rapide et circulaire, un de ces regards de joli garçon, qui s'étendent comme des coups d'épervier.

Les femmes avaient levé la tête vers lui, trois petites ouvrières, une maîtresse de musique entre deux âges, mal peignée, négligée, coiffée d'un chapeau toujours poussiéreux et vêtue d'une robe toujours de travers, et deux bourgeoises avec leurs maris, habituées de cette gargote à prix fixe.

Lorsqu'il fut sur le trottoir, il demeura un instant immobile, se demandant ce qu'il allait faire. On était au 28 juin, et il lui restait juste en poche trois francs quarante pour finir le mois. Cela représentait deux dîners sans déjeuners, ou deux déjeuners sans dîners, au choix. Il ré

**3.  Correction manuelle des annotations en vue de créer un jeu  de données d'entrainement**

Ensuite télécharger fichier json à partir de tagtog et renommez-le en "BelAmi_01_01_corrected.ann.json", ce fichier est traité ci-dessous.

In [19]:
from google.colab import files
uploaded = files.upload()


Saving BelAmi_01_01_corrected.ann.json to BelAmi_01_01_corrected.ann (1).json


**4. Formattage et intégration des annotations corrigées dans Spacy en tant que jeu de données d'entrainement**

In [20]:
# importing the module
import json
  # reading the data from the file
with open('BelAmi_01_01_corrected.ann.json') as f:
    data = f.read()  
print("Data type before reconstruction : ", type(data))
# reconstructing the data as a dictionary
js_corrected_training_data = json.loads(data)
print("Data type after reconstruction : ", type(js_corrected_training_data))
print(js_corrected_training_data)

Data type before reconstruction :  <class 'str'>
Data type after reconstruction :  <class 'dict'>
{'anncomplete': False, 'metas': {}, 'relations': [], 'entities': [{'classId': 'e_1', 'part': 's1v1', 'offsets': [{'start': 95, 'text': 'Georges Duroy'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['ml:fr_core_news_sm'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_2', 'part': 's1v1', 'offsets': [{'start': 1378, 'text': 'rue Notre-Dame-de-Lorette'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['ml:fr_core_news_sm'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_2', 'part': 's1v1', 'offsets': [{'start': 2459, 'text': 'Paris'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['ml:fr_core_news_sm'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_1', 'part': 's1v1', 'offsets': [{'start': 2963, 'text': 'Georges Duroy'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['m

In [21]:
def get_class_id_inv(label):
  """
  Translates the tagtog entity type id into spaCy label id 
  - label: tagtog entity type id
  """
  choices = {'e_1': 'PER', 'e_2': 'LOC', 'e_3': 'MISC'}
  return choices.get(label, None)

In [22]:


training_data = {'classes' : ['PER', "LOC", "MISC"], 'annotations' : []}
temp_dict = {}
temp_dict['text'] = myfile
temp_dict['entities'] = []
for example in js_corrected_training_data['entities']:  
  #print(example)
  start = example['offsets'][0]['start']
  end = int(example['offsets'][0]['start']) + len(example['offsets'][0]['text'])
  label = get_class_id_inv(example['classId'])
  temp_dict['entities'].append((start, end, label))
  #print(temp_dict['entities'])
training_data['annotations'].append(temp_dict)
  
print(training_data['annotations'][0])

{'text': "PREMIÈRE PARTIE.\n\n\n\nI\n\n\n\nQuand la caissière lui eut rendu la monnaie de sa pièce de cent sous, Georges Duroy sortit du restaurant.\n\nComme il portait beau, par nature et par pose d'ancien sous-officier, il cambra sa taille, frisa sa moustache d'un geste militaire et familier, et jeta sur les dîneurs attardés un regard rapide et circulaire, un de ces regards de joli garçon, qui s'étendent comme des coups d'épervier.\n\nLes femmes avaient levé la tête vers lui, trois petites ouvrières, une maîtresse de musique entre deux âges, mal peignée, négligée, coiffée d'un chapeau toujours poussiéreux et vêtue d'une robe toujours de travers, et deux bourgeoises avec leurs maris, habituées de cette gargote à prix fixe.\n\nLorsqu'il fut sur le trottoir, il demeura un instant immobile, se demandant ce qu'il allait faire. On était au 28 juin, et il lui restait juste en poche trois francs quarante pour finir le mois. Cela représentait deux dîners sans déjeuners, ou deux déjeuners sans

**5.  Initialisation du modèle qui sera entrainé de manière incrémentale à partir du jeu d'entrainement** 

In [23]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("fr") # load a new spacy model
doc_bin = DocBin() # create a DocBin object


In [24]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object


100%|██████████| 1/1 [00:00<00:00, 10.48it/s]


Le fichier de configuration base_config_nermodel.cfg est à télécharger à partir d'ici : https://github.com/cvbrandoe/coursTAL/blob/master/2022/base_config_nermodel.cfg 

In [25]:
from google.colab import files
uploaded = files.upload()

Saving base_config_nermodel.cfg to base_config_nermodel.cfg


**6.  Initialisation de l'entrainement avec les paramètres définis (hyperparamètres) dans le fichier .cfg et entrainement par le biais du CLI de Spacy**

In [26]:
!python -m spacy init fill-config base_config_nermodel.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [27]:
!python -m spacy train config.cfg --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --output ./


ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-02-23 10:26:47,623] [INFO] Set up nlp object from config
[2022-02-23 10:26:47,636] [INFO] Pipeline: ['tok2vec', 'ner', 'parser']
[2022-02-23 10:26:47,636] [INFO] Resuming training for: ['ner']
[2022-02-23 10:26:47,649] [INFO] Created vocabulary
[2022-02-23 10:26:47,650] [INFO] Finished initializing nlp object
[2022-02-23 10:26:48,863] [INFO] Initialized pipeline components: ['tok2vec', 'parser']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner', 'parser']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  LOSS PARSER  ENTS_F  ENTS_P  ENTS_R  DEP_UAS  DEP_LAS  SENTS_F  SCORE 
---  ------  ------------  --------  -----------  ------  ------  ------  -------  -------  -------  ------
  0       0          0.00    164.14         0.00   63.51   61.84   65.28     0.00    

Télécharger l'autre extrait de Bel Ami pour tester le modèle à partir de : https://github.com/cvbrandoe/coursTAL/blob/master/2022/belAmi_01-05.txt

In [28]:
from google.colab import files
uploaded = files.upload()

Saving belAmi_01-05.txt to belAmi_01-05.txt


**7. Test du nouveau modèle sur un autre extrait de "Bel Ami" de Maupassant**

In [29]:
nlp_ner = spacy.load("model-best")

test_dataset = open("belAmi_01-05.txt").read().replace("’","'")
doc = nlp_ner(test_dataset)

colors = {"PER": "#F67DE3", "LOC": "#7DF6D9", "MISC" : "#FFFFFF"}
options = {"colors": colors} 

show_ents(doc)

#spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

Duroy - 85 - 90 - PER - Named person or family.


Forestier - 474 - 483 - PER - Named person or family.


Duroy - 660 - 665 - PER - Named person or family.


l'Algérie - 849 - 858 - LOC - Non-GPE locations, mountain ranges, bodies of water


Bois - 1096 - 1100 - LOC - Non-GPE locations, mountain ranges, bodies of water


Mme Forestier - 1730 - 1743 - PER - Named person or family.


Mme de Marelle - 1899 - 1913 - PER - Named person or family.


rue de Verneuil - 2160 - 2175 - LOC - Non-GPE locations, mountain ranges, bodies of water


Duroy - 2425 - 2430 - PER - Named person or family.


Duroy - 3037 - 3042 - PER - Named person or family.


Mme de Marelle - 3115 - 3129 - PER - Named person or family.


Duroy - 3465 - 3470 - PER - Named person or family.


Norbert de Varenne - 3589 - 3607 - PER - Named person or family.


Paris - 3733 - 3738 - LOC - Non-GPE locations, mountain ranges, bodies of water


Mme Forestier - 4599 - 4612 - PER - Named person or family.


Mme de Marelle - 5018 - 

**A titre de comparaison, test avec ce même texte mais à partir du modèle de base fourni par Spacy, fr_core_news_sm.**

In [30]:
pipeline = 'fr_core_news_sm' 
nlp = spacy.load(pipeline)
doc = nlp(test_dataset)
show_ents(doc)

V



 - 0 - 5 - PER - Named person or family.


Duroy - 85 - 90 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


Forestier - 474 - 483 - PER - Named person or family.


Duroy - 660 - 665 - LOC - Non-GPE locations, mountain ranges, bodies of water


l'Algérie - 849 - 858 - LOC - Non-GPE locations, mountain ranges, bodies of water


Bois - 1096 - 1100 - LOC - Non-GPE locations, mountain ranges, bodies of water


Mme Forestier - 1730 - 1743 - PER - Named person or family.


Mme de Marelle - 1899 - 1913 - PER - Named person or family.


rue de Verneuil - 2160 - 2175 - LOC - Non-GPE locations, mountain ranges, bodies of water


madame - 2310 - 2316 - PER - Named person or family.


Mme de Marelle - 3115 - 3129 - PER - Named person or family.


Duroy - 3465 - 3470 - LOC - Non-GPE locations, mountain ranges, bodies of water


Norbert de Varenne - 3589 - 3607 - PER - Named person or family.


Paris - 3733 - 3738 - LOC - Non-GPE locations, mountain ranges,

Remarque 1. lors de la mise à jour du modèle NER (idem pour d'autres couches comme POS) avec les annotations de l'utilisateur, un processus itératif se met en place afin de faire converger les prédictions faites (des poids) vers des valeurs optimales vis-à-vis des annotations de référence. A caque itération, on calcule le "gradient" qui correspond, en optimisation, aux dérivées partielles de la fonction de classification non linéaire recherchée (voir [1], vous trouverez aussi la notion de backpropagation clé dans ces approches de DL pour le TAL). Pour observer la manière dont la valeur du gradient évolue (et se stabilise) lors des itérations, il est conseillé de regarder le paramètre "losses" mis à jour par la méthode update(..., losses=losses).
[1]: https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1162/handouts/CS224N_DeepNLP_Week7_lecture3.pdf 
Aussi : https://spacy.io/usage/training

Remarque 2. Pour éviter le surapprentissage (overfitting) d'un modèle : vous pouvez identifier que votre modèle n'est pas bon lorsqu'il fonctionne bien sur les données d'entraînement mais ne donne pas de bons résultats sur des données nouvelles et non encore vues. Autrement dit, le modèle "mémorise" les données d'apprentissage et n'est pas performant avec les nouvelles données.